In [ ]:
import pandas as pd
import numpy as np
import plotly as plt

import math
import scipy

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import geopandas as gpd

import plotly.io as pio
import plotly.graph_objects as go   
import country_converter as coco
pio.renderers.default = "plotly_mimetype+notebook"

In [ ]:
freight = pd.read_csv("Data_Sets/avia_gor_nl__custom_3564729_monthly_linear.csv")
passengers = pd.read_csv("Data_Sets/avia_par_nl__custom_3564728_monthly_linear.csv")
country_code_fix = {'EL':'GR','AN':'BQ'}

def top_n_year(df,n,year):
    df[["airp_country_1","airp_code_1","airp_country_2","airp_code_2"]] = df.airp_pr.str.split("_",expand=True)
    df[["Year","Month"]] = df.TIME_PERIOD.str.split("-",expand=True)
    topn = df.groupby(['Year',"airp_country_2"]).sum()
    topn = topn.loc[year,:].sort_values("OBS_VALUE", ascending= False).head(n).reset_index()
    cleaned = df[df["airp_country_2"].isin(topn["airp_country_2"])]
    return cleaned

def codes_correction(df,ISO_2):
    df[ISO_2] = df[ISO_2].replace(to_replace= country_code_fix)
    df['iso_3_country'] = coco.convert(names = df[ISO_2], to= 'ISO3')
    df['country_2_name'] = coco.convert(names = df[ISO_2], to= 'name_short')
    return df

top_passengers = top_n_year(passengers,5,'2019')
top_cargo = top_n_year(freight,5,'2019')

In [ ]:
total_cargo = top_cargo.groupby(['airp_country_2','Year']).sum()
total_cargo = total_cargo.reset_index()
total_cargo = codes_correction(total_cargo,'airp_country_2')
fig = px.line(total_cargo, x='Year', y= 'OBS_VALUE' ,color='airp_country_2')
fig.show()

In [ ]:
total_passengers = top_passengers.groupby(['airp_country_2','Year']).sum()
total_passengers = total_passengers.reset_index()
total_passengers = codes_correction(total_passengers,'airp_country_2')
fig = px.line(total_passengers, x='Year', y= 'OBS_VALUE' ,color='airp_country_2')
fig.show()

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head()

In [ ]:
passengers_for_map = passengers.groupby(['Year','airp_country_2']).sum()
passengers_for_map = passengers_for_map.loc['2019',:].reset_index()
passengers_for_map = codes_correction(passengers_for_map,'airp_country_2')

freight_for_map = freight.groupby(['Year','airp_country_2']).sum()
freight_for_map = freight_for_map.loc['2019',:].reset_index()
freight_for_map = codes_correction(freight_for_map,'airp_country_2')


In [ ]:
map_data_passengers = world.merge(passengers_for_map,left_on = 'iso_a3', right_on = 'iso_3_country', how = 'left')

fig = px.choropleth(map_data_passengers, hover_name = 'country_2_name',locations= 'iso_3_country', locationmode= 'ISO-3', color='OBS_VALUE')

fig.update_layout(
    title_text = 'Total passengers flying from Schiphol Airport in 2019',
    geo= dict(
        showframe = False,
        showcoastlines = False,
        projection_type = 'equirectangular'
    )
)
fig.show()

In [ ]:
map_data_cargo = world.merge(freight_for_map,left_on = 'iso_a3', right_on = 'iso_3_country', how = 'left')

fig = px.choropleth(freight_for_map, hover_name = 'country_2_name',locations= 'iso_3_country', locationmode= 'ISO-3', color='OBS_VALUE')

fig.update_layout(
    title_text = 'Total tons transported by air from Schiphol Airport in 2019',
    geo= dict(
        showframe = False,
        showcoastlines = False,
        projection_type = 'equirectangular'
    )
)
fig.show()

In [ ]:
fig = px.bar(map_data_passengers.sort_values('OBS_VALUE',ascending= False).head(20),x= 'country_2_name', y = 'OBS_VALUE')
fig.show()

In [ ]:
fig = px.bar(map_data_cargo.sort_values('OBS_VALUE',ascending= False).head(20),x= 'country_2_name', y = 'OBS_VALUE')
fig.show()